<a href="https://colab.research.google.com/github/Adaline-K/git4-collaboration-sourcetree/blob/master/tensortest7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#학습이 잘 되는 모델구현(Bach Normalization)
배치 정규화란 활성화함수의 ㅣ활성화값 또는 출력값을 정규화(정규분포)하는 작업


1.   학습 속도가 개선됨(학습률을 높게 설정할 수 있기 때문)
2.   가중치 초깃값 선택의 의존성이 적어짐(학습을 할때마다 출력값을 정규화하기 때문)
3.   과적합(Overfitting)위험을 줄일 수 있음
4.   Gradient Vanishing 문제해결



In [2]:
#라이브러리 사용
import tensorflow as tf
import pandas as pd

In [3]:
#보스턴 집값 예측
#과거 데이터 준비
파일경로 = 'https://raw.githubusercontent.com/blackdew/tensorflow1/master/csv/boston.csv'
보스턴 = pd.read_csv(파일경로)
print(보스턴.columns)

#종속, 독립변수
독립 = 보스턴[['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax',
       'ptratio', 'b', 'lstat']]
종속 = 보스턴[['medv']]
print(독립.shape, 종속.shape)

Index(['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax',
       'ptratio', 'b', 'lstat', 'medv'],
      dtype='object')
(506, 13) (506, 1)


In [5]:
#모델구조만들기
X = tf.keras.layers.Input(shape=[13])

H = tf.keras.layers.Dense(8, activation='swish')(X)
H = tf.keras.layers.Dense(8, activation='swish')(H)
H = tf.keras.layers.Dense(8, activation='swish')(H)

Y = tf.keras.layers.Dense(1)(H)
model = tf.keras.models.Model(X,Y)
model.compile(loss='mse')

In [8]:
#모델의 구조를 BachNormalization layer를 사용하여 만든다
X = tf.keras.layers.Input(shape=[13])

H = tf.keras.layers.Dense(8)(X)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('swish')(H)

H = tf.keras.layers.Dense(8)(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('swish')(H)

H = tf.keras.layers.Dense(8)(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('swish')(H)

Y = tf.keras.layers.Dense(1)(H)
model = tf.keras.models.Model(X,Y)
model.compile(loss='mse')

In [9]:
#모델구조확인
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 13)]              0         
_________________________________________________________________
dense_10 (Dense)             (None, 8)                 112       
_________________________________________________________________
batch_normalization_3 (Batch (None, 8)                 32        
_________________________________________________________________
activation_3 (Activation)    (None, 8)                 0         
_________________________________________________________________
dense_11 (Dense)             (None, 8)                 72        
_________________________________________________________________
batch_normalization_4 (Batch (None, 8)                 32        
_________________________________________________________________
activation_4 (Activation)    (None, 8)                 0   

In [25]:
#데이터로 모델학습시키기(FIT)
model.fit(독립, 종속, epochs=1000, verbose=0)
model.fit(독립, 종속, epochs=10)


Epoch 1/10
5/5 [==============================] - 0s 2ms/step - loss: 0.0451 - accuracy: 0.9800
Epoch 2/10
5/5 [==============================] - 0s 2ms/step - loss: 0.0808 - accuracy: 0.9667
Epoch 3/10
5/5 [==============================] - 0s 2ms/step - loss: 0.0364 - accuracy: 0.9933
Epoch 4/10
5/5 [==============================] - 0s 2ms/step - loss: 0.0484 - accuracy: 0.9733
Epoch 5/10
5/5 [==============================] - 0s 2ms/step - loss: 0.0437 - accuracy: 0.9800
Epoch 6/10
5/5 [==============================] - 0s 3ms/step - loss: 0.0384 - accuracy: 0.9800
Epoch 7/10
5/5 [==============================] - 0s 2ms/step - loss: 0.0987 - accuracy: 0.9600
Epoch 8/10
5/5 [==============================] - 0s 2ms/step - loss: 0.0510 - accuracy: 0.9800
Epoch 9/10
5/5 [==============================] - 0s 2ms/step - loss: 0.0301 - accuracy: 0.9933
Epoch 10/10
5/5 [==============================] - 0s 2ms/step - loss: 0.0587 - accuracy: 0.9600


In [26]:
#모델이용
print(model.predict(독립[0:5]))
print(종속[0:5])


[[1.00000000e+00 5.35248512e-11 5.65992396e-14]
 [1.00000000e+00 2.06715778e-09 1.42697952e-12]
 [1.00000000e+00 2.69590444e-11 1.26069995e-14]
 [1.00000000e+00 1.14850307e-09 5.06992668e-13]
 [1.00000000e+00 1.35087531e-11 1.02586888e-14]]
   품종_setosa  품종_versicolor  품종_virginica
0          1              0             0
1          1              0             0
2          1              0             0
3          1              0             0
4          1              0             0


In [14]:
#아이리스 품종 분류
#과거의 데이터 준비
파일경로 = 'https://raw.githubusercontent.com/blackdew/tensorflow1/master/csv/iris.csv'
아이리스 = pd.read_csv(파일경로)

#원핫인코딩
아이리스 = pd.get_dummies(아이리스)
print(아이리스.columns)
#독립, 종속변수
독립 = 아이리스[['꽃잎길이', '꽃잎폭', '꽃받침길이', '꽃받침폭']]
종속 = 아이리스[['품종_setosa', '품종_versicolor',
       '품종_virginica']]
print(독립.shape, 종속.shape)

Index(['꽃잎길이', '꽃잎폭', '꽃받침길이', '꽃받침폭', '품종_setosa', '품종_versicolor',
       '품종_virginica'],
      dtype='object')
(150, 4) (150, 3)


In [15]:
#모델구조 만들기
X = tf.keras.layers.Input(shape=[4])
H = tf.keras.layers.Dense(8, activation='swish')(X)
H = tf.keras.layers.Dense(8, activation='swish')(H)
H = tf.keras.layers.Dense(8, activation='swish')(H)
Y = tf.keras.layers.Dense(3, activation='softmax')(H)
model = tf.keras.models.Model(X,Y)
model.compile(loss='categorical_crossentropy', metrics='accuracy')

In [18]:
#모델구조를 BachNormalizaiton layer를 사용하여 만든다.
X = tf.keras.layers.Input(shape=[4])

H = tf.keras.layers.Dense(8)(X)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('swish')(H)

H = tf.keras.layers.Dense(8)(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('swish')(H)

H = tf.keras.layers.Dense(8)(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('swish')(H)

Y = tf.keras.layers.Dense(3, activation='softmax')(H)
model = tf.keras.models.Model(X,Y)
model.compile(loss='categorical_crossentropy', metrics='accuracy')

In [19]:
#모델구조 확인
model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_23 (Dense)             (None, 8)                 40        
_________________________________________________________________
batch_normalization_6 (Batch (None, 8)                 32        
_________________________________________________________________
activation_9 (Activation)    (None, 8)                 0         
_________________________________________________________________
dense_24 (Dense)             (None, 8)                 72        
_________________________________________________________________
batch_normalization_7 (Batch (None, 8)                 32        
_________________________________________________________________
activation_10 (Activation)   (None, 8)                 0   

In [23]:
#데이터로 모델 학습시키기(FIT)
model.fit(독립, 종속, epochs=100, verbose=0)
model.fit(독립, 종속, epochs=10)

Epoch 1/10
5/5 [==============================] - 0s 3ms/step - loss: 0.0745 - accuracy: 0.9667
Epoch 2/10
5/5 [==============================] - 0s 2ms/step - loss: 0.0486 - accuracy: 0.9933
Epoch 3/10
5/5 [==============================] - 0s 2ms/step - loss: 0.0790 - accuracy: 0.9667
Epoch 4/10
5/5 [==============================] - 0s 3ms/step - loss: 0.1455 - accuracy: 0.9333
Epoch 5/10
5/5 [==============================] - 0s 2ms/step - loss: 0.0642 - accuracy: 0.9667
Epoch 6/10
5/5 [==============================] - 0s 2ms/step - loss: 0.0817 - accuracy: 0.9733
Epoch 7/10
5/5 [==============================] - 0s 2ms/step - loss: 0.1353 - accuracy: 0.9600
Epoch 8/10
5/5 [==============================] - 0s 2ms/step - loss: 0.1179 - accuracy: 0.9333
Epoch 9/10
5/5 [==============================] - 0s 2ms/step - loss: 0.0730 - accuracy: 0.9800
Epoch 10/10
5/5 [==============================] - 0s 2ms/step - loss: 0.0613 - accuracy: 0.9867


In [24]:
#모델이용
print(model.predict(독립[0:5]))
print(종속[0:5])

[[9.9999642e-01 3.2626447e-06 3.7711564e-07]
 [9.9997532e-01 2.1508578e-05 3.2341345e-06]
 [9.9999702e-01 2.5930995e-06 3.9138672e-07]
 [9.9997520e-01 2.1989321e-05 2.8975801e-06]
 [9.9999797e-01 1.8351967e-06 1.9840114e-07]]
   품종_setosa  품종_versicolor  품종_virginica
0          1              0             0
1          1              0             0
2          1              0             0
3          1              0             0
4          1              0             0
